## Objective

In this notebook we:

In this notebook we:

- Load the **model-ready** Telco churn dataset
- Split the data into **train** and **test** sets (stratified by churn)
- Train and compare several models:
  - Logistic Regression (with feature scaling)
  - Decision Tree 
  - Random Forest 
  - Neural Network
  - XGBoost / Gradient Boosting
- Perform **light hyperparameter tuning** with GridSearchCV
- Evaluate models using:
  - Accuracy, Precision, Recall, F1-score
  - ROC AUC
  - Confusion matrix
  - ROC Curve
- Test the **impact of engineered features**:
  - NumServices
  - TenureGroup_* one-hot columns
- Interpret the final model using:
  - Feature importance (Random Forest)
  - SHAP (global importance and local explanations)

We are **not assuming** a single model family upfront.  
Instead, we compare **linear** (Logistic Regression) and **tree-based** models and select the one that performs best and is most interpretable.

## 1. Imports and Data Loading

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
    RocCurveDisplay,
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
# Paths
NOTEBOOK_DIR = Path.cwd()
MODEL_DATA = (NOTEBOOK_DIR / "../data/model_ready").resolve()

data_path = MODEL_DATA / "telco_churn_model_ready.csv"
df = pd.read_csv(data_path)

print("Loaded model-ready dataset:", df.shape)
df.head()

Loaded model-ready dataset: (7043, 44)


,Age,Senior Citizen,Married,Dependents,Number of Dependents,Referred a Friend,Number of Referrals,Tenure in Months,Phone Service,Avg Monthly Long Distance Charges,...,Internet Type_No Internet,Contract_One Year,Contract_Two Year,Payment Method_Credit Card,Payment Method_Mailed Check,TenureGroup_6–12,TenureGroup_12–24,TenureGroup_24–48,TenureGroup_48–72,Churn Value
0,78,1,0,0,0,0,0,1,0,0.00,...,False,False,False,False,False,False,False,False,False,1
1,74,1,1,1,1,1,1,8,1,48.85,...,False,False,False,True,False,True,False,False,False,1
2,71,1,0,1,3,0,0,18,1,11.33,...,False,False,False,False,False,False,True,False,False,1
3,78,1,1,1,1,1,1,25,1,19.76,...,False,False,False,False,False,False,False,True,False,1
4,80,1,1,1,1,1,1,37,1,6.33,...,False,False,False,False,False,False,False,True,False,1


## 2. Basic checks and train/test split

We separate:

- `y` = target (`Churn Value`)
- `X` = all remaining features

Then we create a **stratified** train/test split to keep the churn rate similar in both sets.

### Why We Use Stratified Train/Test Split

In this dataset, the target variable (Churn Value) is highly imbalanced:

- ~73% of customers do not churn
- ~27% of customers churn

If we were to split the dataset randomly without stratification, we could easily end up with:

- Training set with too few churners
- Test set with many more churners (or vice-versa)

This creates serious problems:

1. Model learns a biased view of churn

If the training set has fewer churners than the real distribution, the model may learn:

- “Almost everyone stays.”

This leads to high accuracy but terrible recall, which is dangerous for churn prediction.

2. Test performance becomes unstable

If the test set has a different churn ratio from the training set:

- Evaluation metrics become misleading
- ROC AUC becomes unreliable
- Models may look good only because the test set is easier

3. Cross-model comparisons become unfair

Models trained on different random splits can behave very differently
if the churn distribution varies.

In [3]:
target = "Churn Value"

X = df.drop(columns=[target])
y = df[target]

print("Feature matrix shape:", X.shape)
print("Target shape:", y.shape)

Feature matrix shape: (7043, 43)
Target shape: (7043,)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

print("Train shapes:", X_train.shape, y_train.shape)
print("Test shapes:", X_test.shape, y_test.shape)
print("Churn rate - overall:", y.mean().round(3))
print("Churn rate - train:", y_train.mean().round(3))
print("Churn rate - test:", y_test.mean().round(3))

Train shapes: (5634, 43) (5634,)
Test shapes: (1409, 43) (1409,)
Churn rate - overall: 0.265
Churn rate - train: 0.265
Churn rate - test: 0.265


### Scaling and model families

We will compare two main families of models:

- **Linear models** (Logistic Regression, Neural Network input layer):
  - Sensitive to feature scale.
  - We use `StandardScaler` inside a `Pipeline`.

- **Tree-based models** (Decision Tree, Random Forest, XGBoost):
  - **Do not** require feature scaling.
  - Work directly with the raw feature scales and thresholds.
  - Perform implicit feature selection and handle non-linearity.

This is why we **did not scale features in the Feature Engineering notebook**.  
Scaling is applied only **inside** pipelines for the models that actually need it,  
keeping the dataset clean and flexible for all model types.


## 3. Evaluation helper

Instead of repeating the same evaluation code for every model, we create one helper function that:

- trains the model  
- generates predictions  
- computes key metrics (accuracy, precision, recall, F1, ROC AUC)  
- plots the confusion matrix and ROC curve  

This ensures **all models are evaluated consistently and fairly**, using the exact same process. It keeps the notebook clean, avoids duplicated code, and makes model comparison straightforward.

In [5]:
def evaluate_model(name, model, X_train, X_test, y_train, y_test, verbose=True, plot=True):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # For ROC AUC, use predicted probabilities if available
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]
    elif hasattr(model, "decision_function"):
        y_proba = model.decision_function(X_test)
    else:
        y_proba = None

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    auc = roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan

    if verbose:
        print(f"=== {name} ===")
        print(f"Accuracy : {acc:.3f}")
        print(f"Precision: {prec:.3f}")
        print(f"Recall   : {rec:.3f}")
        print(f"F1-score : {f1:.3f}")
        print(f"ROC AUC  : {auc:.3f}")
        print("\nClassification report:\n")
        print(classification_report(y_test, y_pred, zero_division=0))

    if plot and y_proba is not None:
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=axes[0])
        axes[0].set_title(f"{name} - Confusion Matrix")
        axes[0].set_xlabel("Predicted")
        axes[0].set_ylabel("Actual")

        # ROC curve
        RocCurveDisplay.from_predictions(y_test, y_proba, ax=axes[1])
        axes[1].set_title(f"{name} - ROC Curve")

        plt.tight_layout()
        plt.show()

    return {
        "model": name,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "roc_auc": auc,
    }